go/meterstick-demo-aqlogs

In [ ]:
from colabtools import adhoc_import

try:
  from google3.ads.metrics.lib.development.datacube import datacube_colab
except ImportError:
  with adhoc_import.Google3SubmittedChangelist():
    from google3.ads.metrics.lib.development.datacube import datacube_colab

datacube_colab.setup()

with adhoc_import.Google3():
  from meterstick.v2 import *

Done! Processed commands DEFINE: 6, LOAD: 0, SET: 0


Done! Processed commands DEFINE: 1136, LOAD: 0, SET: 0


In [ ]:
%%dremel
SET SQL_DIALECT GOOGLESQL;
SET MIN_COMPLETION_RATIO 1;

Done! Processed commands DEFINE: 0, LOAD: 0, SET: 3


In [ ]:
table = """SELECT
  $Experiment((2004375, 2004376)) as experiment,
  $CookieBucket AS cookie,
  $PropertyGroup AS property_group,
  SUM($Queries) AS queries_ads,
  SUM($SessionQueryEvents) AS queries_event,
  SUM($SumOverImpressions($Clicks)) AS clicks
FROM $DatedQueryTable(20200901)
WHERE $PropertyGroup IN ("google", "mobile_highend", "google_tablet")
GROUP BY 1, 2, 3"""

queries_ads = Sum('queries_ads', 'queries_ads')
queries_event = Sum('queries_event', 'queries_event')
metrics = MetricList((queries_ads, queries_event))
res = (metrics
       | PercentChange('experiment', 2004375)
       | Jackknife('cookie', confidence=0.95)
       | compute_on_sql(table, 'property_group'))
res

Metric                    queries_ads_percent_change  ... queries_event_percent_change
                                               Value  ...           Jackknife CI-upper
property_group experiment                             ...                             
google         2004376                    300.316363  ...                   300.863995
google_tablet  2004376                    301.177452  ...                   302.383974
mobile_highend 2004376                    300.057285  ...                   300.421654

[3 rows x 6 columns]

In [ ]:
res.display()

Agrees to [Rasta](https://rasta.corp.google.com/#/metrics?snapshot=xunmo:ShoppingTuningLayerCookieDayControl_20200921_163946).

![Rasta](https://screenshot.googleplex.com/7pHvRPDYF8Kub6u.png)

In [ ]:
mh_ctr = (Ratio('clicks', 'queries_ads', 'ctr')
       | MH('experiment', 2004375, 'property_group')
       | Jackknife('cookie', confidence=0.95)
       | compute_on_sql(table))
mh_ctr.display()